In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 2.8 MB 11.2 MB/s 
     |████████████████████████████████| 1.2 MB 62.0 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 636 kB 56.1 MB/s 
     |████████████████████████████████| 895 kB 76.3 MB/s 
     |████████████████████████████████| 3.3 MB 53.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=e4eaed372cfa59c55eb30db582c0287d434385f1fc4135f843c5135f4afac82c
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

In [ ]:
# Define your sentence transformer model using CLS pooling
model_name = "xlnet-base-cased"
word_embedding_model = models.Transformer(model_name, max_seq_length = 512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/train_second_batch.csv', index_col = 0)

In [ ]:
df

,id,url_legal,license,excerpt,target,standard_error,top_pop,mean_word_length,sent_lengths,word_count,num_nouns,num_verbs,num_adjectives
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,31,4.988764,464,74,490,88,68
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,23,4.823529,426,72,500,97,59
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,20,4.790698,430,74,496,87,64
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,36,4.858696,474,80,481,88,61
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,10,4.394366,123,23,197,31,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900,19,5.534091,368,59,452,88,63
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648,19,5.024096,347,58,405,79,56
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866,14,5.011905,313,52,369,83,64
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128,18,5.418919,381,58,430,74,63


In [ ]:
# Define a list with sentences (1k - 100k sentences)
train_sentences = list(df.excerpt)

a = [i.split('.') for i in train_sentences]
import itertools
a = list(itertools.chain(*a))
train_sentences = list(filter(None, a))

In [ ]:
# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

In [ ]:
# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.


In [ ]:
# total_steps = 2000
# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True)

model.save('output/tsdae-model')
model.save('/content/drive/MyDrive/tsdae-model')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3241 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/xlnet/modeling_xlnet.py:1104: FutureWarning: The `use_cache` argument is deprecated and will be removed in a future version, use `use_mems` instead.
  FutureWarning,


In [ ]:
model = SentenceTransformer(model_name_or_path= '/content/drive/MyDrive/tsdae-model')

In [ ]:
model.encode(['Hi, my name is Abhilash'])

array([[-4.5994034e-01, -2.5478082e+00, -1.0732641e+00,  2.7938406e+00,
        -1.0504983e-01,  3.8302767e+00,  6.4530426e-01,  4.5534244e+00,
         2.1199458e+00,  2.5345438e+00, -3.1439078e-01, -1.3308308e+00,
        -1.2038602e+00, -1.9781085e+00,  1.0786378e+00, -4.0027637e+00,
         4.4970798e+00,  5.3761601e-01,  8.7053254e-03,  1.2080678e+00,
        -1.1219399e-01,  1.2792631e+00,  3.1482232e-01,  1.8885450e+00,
         3.0985795e-02,  3.3543658e-01, -1.4551523e+00,  5.2013963e-01,
         3.4822397e+00, -2.1366456e+00,  1.6601844e+00,  1.8555197e+00,
         6.5366192e+00,  2.1381469e+00, -1.4834013e+00,  1.4529092e+00,
         3.0713420e+00,  8.2950604e-01,  1.3936160e+00, -1.2285237e+00,
         2.2434359e+00,  8.0902535e-01,  3.2991204e-01,  1.4245511e+00,
         1.3289332e+00,  2.2349248e+00,  2.9250119e+00, -2.0190887e+00,
        -3.0833211e+00,  1.9726990e+00,  3.3095920e+00, -3.7383494e+00,
         1.2633163e+00,  7.1545976e-01, -6.4413691e-01,  1.96822